# QIIME 2 enables comprehensive end-to-end analysis of diverse microbiome data and comparative studies with publicly available data

this is a QIIME 2 Artifact CLI notebook which replicated analyses in the QIIME 2 protocol

**environment:** qiime2-2020.2

In [49]:
pip install --upgrade c-lasso

Processing /Users/lsimpson/Library/Caches/pip/wheels/05/4a/e3/9c53d5ed27403a785368afcae55a715603472a82bb66faf6f2/c_lasso-1.0.0.5-py3-none-any.whl
  Attempting uninstall: c-lasso
    Found existing installation: c-lasso 1.0.0.4
    Uninstalling c-lasso-1.0.0.4:
      Successfully uninstalled c-lasso-1.0.0.4


In [ ]:
pip install c-lasso

pip install zarr

pip install plotly

In [96]:
cd ..

In [97]:
ls

README.md		example			setup.py
build			q2_classo		tutorial
dist			q2_classo.egg-info


In [98]:
python setup.py install

pip install -e .

running install
running bdist_egg
running egg_info
writing q2_classo.egg-info/PKG-INFO
writing dependency_links to q2_classo.egg-info/dependency_links.txt
writing entry points to q2_classo.egg-info/entry_points.txt
writing top-level names to q2_classo.egg-info/top_level.txt
reading manifest file 'q2_classo.egg-info/SOURCES.txt'
writing manifest file 'q2_classo.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-x86_64/egg
running install_lib
running build_py
copying q2_classo/_func.py -> build/lib/q2_classo
copying q2_classo/plugin_setup.py -> build/lib/q2_classo
copying q2_classo/_tree.py -> build/lib/q2_classo
copying q2_classo/_summarize/_visualizer.py -> build/lib/q2_classo/_summarize
copying q2_classo/_summarize/assets/cv.html -> build/lib/q2_classo/_summarize/assets
copying q2_classo/_summarize/assets/lam-fixed.html -> build/lib/q2_classo/_summarize/assets
copying q2_classo/_summarize/assets/stabsel.html -> build/lib/q2_classo/_summarize/assets
creating build

In [99]:
qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [100]:
cd example/Data

## Upload data

## Taxonomic classification

## log-contrast and taxa processing

In [11]:
qiime taxa collapse --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table genus_table.qza

qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features genus_table.qza \
     --o-x genus_table_clr
     
 qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features table.qza \
     --o-x xclr
     
    
qiime classo add-taxa \
	--i-features xclr.qza  \
	--i-taxa taxonomy.qza \
	--o-x xtaxa --o-aweights wtaxa



Saved FeatureTable[Frequency] to: genus_table.qza
Saved FeatureTable[Design] to: genus_table_clr.qza
Saved FeatureTable[Design] to: xclr.qza
Saved FeatureTable[Design] to: xtaxa.qza
Saved Weights to: wtaxa.qza


## Add covariates

In [125]:
qiime classo add-covariates \
    --i-features xtaxa.qza \
    --i-weights wtaxa.qza \
    --m-covariates-file sample-metadata-complete.tsv \
    --p-to-add host_sexual_orientation host_age host_body_mass_index ethnicity \
    --p-w-to-add 0.5 0.5 0.5 0.5 \
    --o-new-features xcovariates \
    --o-new-c ccovariates \
    --o-new-w wcovariates \
    --verbose

Saved FeatureTable[Design] to: xcovariates.qza
Saved ConstraintMatrix to: ccovariates.qza
Saved Weights to: wcovariates.qza


## Split table

Split data into training and testing sets : 

In [126]:
qiime sample-classifier split-table \
	--i-table xcovariates.qza \
	--m-metadata-file sample-metadata-complete.tsv \
	--m-metadata-column sCD14  \
	--p-test-size 0.2 \
	--p-random-state 123 \
	--p-stratify False \
	--o-training-table xtraining \
	--o-test-table xtest

Saved FeatureTable[Design] to: xtraining.qza
Saved FeatureTable[Design] to: xtest.qza


## Regression task 

Apply classo to the training set to solve the linear regression problem : 

In [127]:
qiime classo regress  \
    --i-features xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column sCD14  \
    --p-concomitant False \
    --p-stabsel False \
    --p-cv True \
    --p-path True \
    --p-lamfixed True \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 1 \
    --p-cv-one-se False \
    --o-result regresstaxa

Saved CLASSOProblem to: regresstaxa.qza


## Classification task

In [128]:
qiime classo classify  \
    --i-features xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column HIV_serostatus  \
    --p-stabsel False \
    --p-cv True \
    --p-path True \
    --p-lamfixed True \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 1 \
    --p-cv-one-se False \
    --o-result classifytaxa

Saved CLASSOProblem to: classifytaxa.qza


## Prediction 

In [129]:
qiime classo predict \
    --i-features xtest.qza \
    --i-problem regresstaxa.qza \
    --o-predictions regress-predictions.qza

Saved CLASSOProblem to: regress-predictions.qza


In [130]:
qiime classo predict \
    --i-features xtest.qza \
    --i-problem classifytaxa.qza \
    --o-predictions classify-predictions.qza

Saved CLASSOProblem to: classify-predictions.qza


## Visualization

In [131]:
qiime classo summarize \
  --i-problem regresstaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions regress-predictions.qza \
  --o-visualization regresstaxa.qzv

Saved Visualization to: regresstaxa.qzv


In [132]:
qiime classo summarize \
  --i-problem classifytaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions classify-predictions.qza \
  --o-visualization classifytaxa.qzv

Saved Visualization to: classifytaxa.qzv


In [133]:
qiime tools view regresstaxa.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [134]:
qiime tools view classifytaxa.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Alternatively, one can drag&drop the file problemtaxa.qzv on : https://view.qiime2.org
Thanks to this alternative, one can also track the workflow that the qiime2 artifact did. 